In [1]:
!pip install selenium chromedriver-py tqdm pandas requests


In [15]:
# Import necessary libraries
import time
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from chromedriver_py import binary_path
import pandas as pd
import requests

# Set global variables
max_page = 300  
sleep_time = 1  
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("window-size=1900,800")

def get_browser():
    service = Service(executable_path=binary_path)
    wd = webdriver.Chrome(service=service, options=chrome_options)
    return wd

browser = get_browser()

# Navigate to TED Talks page
def talks_page():
    url = 'https://www.ted.com/talks?sort=newest'
    print(f'Navigating to {url}')
    browser.get(url)
    time.sleep(sleep_time * 4)
    try:
        cookie_btn = browser.find_element(By.ID, 'onetrust-accept-btn-handler')
        cookie_btn.click()
    except:
        pass
    time.sleep(sleep_time)

talks_page()

# Scrape TED Talks metadata
final = []
for page in tqdm(range(0, max_page)):
    payload = [
        {"indexName": "newest",
         "params": {"attributeForDistinct": "objectID",
                    "distinct": 1,
                    "facets": ["subtitle_languages", "tags"],
                    "highlightPostTag": "__/ais-highlight__",
                    "highlightPreTag": "__ais-highlight__",
                    "hitsPerPage": 24,
                    "maxValuesPerFacet": 500,
                    "page": page, "query": "",
                    "tagFilters": ""}
         }
    ]
    r = requests.post('https://zenith-prod-alt.ted.com/api/search',
                      headers={'Content-type': 'application/json; charset=UTF-8',
                               "User-Agent": "curl/7.64.1"},
                      json=payload)
    if r.status_code == 200:
        my_tedx = r.json()['results'][0]["hits"]
        final.extend(my_tedx)
    else:
        print(f"Error: {r.status_code} on page {page}")
    time.sleep(sleep_time)

# Extract main metadata fields
final_list = []
for talk in final:
    slug = talk["slug"]
    final_list.append({
        'id': talk["objectID"],
        'slug': talk["slug"],
        'speakers': talk["speakers"],
        'title': talk["title"],
        'url': f'https://www.ted.com/talks/{slug}'
    })

print(f"Total talks scraped: {len(final_list)}")

# Fetch additional details, images, and related data
details = []
images = []
tags = []
related_videos = []
ready = []

for video in tqdm(final_list):
    slug = video["slug"]
    if slug not in ready:
        query = f"""
          {{video(slug: "{slug}", language: "en")
            {{description
              socialDescription
              duration
              id
              presenterDisplayName
              primaryImageSet {{url}}
              publishedAt
              topics {{nodes {{name}}}}
              relatedVideos {{
                id slug title duration viewedCount presenterDisplayName
              }}
            }}
          }}
        """
        payload = [{"operationName": None, "variables": {}, "query": query}]
        r = requests.post('https://www.ted.com/graphql',
                          headers={'Content-type': 'application/json; charset=UTF-8',
                                   "User-Agent": "curl/7.64.1"},
                          json=payload)
        data = r.json()
        if len(data) > 0 and data[0].get("data", {}).get("video") is not None:
            video_data = data[0]["data"]["video"]
            details.append({
                "id": video["id"],
                "slug": video["slug"],
                "internalId": video_data["id"],
                "description": video_data["description"],
                "duration": video_data["duration"],
                "presenterDisplayName": video_data["presenterDisplayName"],
                "publishedAt": video_data["publishedAt"]
            })
            if "primaryImageSet" in video_data:
                for image in video_data["primaryImageSet"]:
                    images.append({"id": video["id"], "url": image["url"]})
            if "topics" in video_data:
                for topic in video_data["topics"]["nodes"]:
                    tags.append({"id": video["id"], "tag": topic["name"]})
            if "relatedVideos" in video_data:
                for related in video_data["relatedVideos"]:
                    related_videos.append({
                        "id": video["id"],
                        "related_id": related["id"],
                        "slug": related["slug"],
                        "title": related["title"],
                        "presenterDisplayName": related["presenterDisplayName"]
                    })
        ready.append(slug)

# Save data to CSV files
print("Saving data to CSV files...")
pd.DataFrame(final_list).to_csv('ted_talks_list.csv', index=False)
pd.DataFrame(details).to_csv('ted_talks_details.csv', index=False)
pd.DataFrame(images).to_csv('ted_talks_images.csv', index=False)
pd.DataFrame(tags).to_csv('ted_talks_tags.csv', index=False)
pd.DataFrame(related_videos).to_csv('ted_talks_related_videos.csv', index=False)

print("Scraping completed! Data saved to CSV files.")        


2024-12-06 00:33:26,263 - DEBUG - Skipping Selenium Manager; path to chrome driver specified in Service class: C:\Users\sathu\anaconda3\Lib\site-packages\chromedriver_py\chromedriver_win64.exe
2024-12-06 00:33:26,277 - DEBUG - Started executable: `C:\Users\sathu\anaconda3\Lib\site-packages\chromedriver_py\chromedriver_win64.exe` in a child process with pid: 56904 using 0 to output -3
2024-12-06 00:33:26,788 - DEBUG - Starting new HTTP connection (1): localhost:49279
2024-12-06 00:33:27,339 - DEBUG - http://localhost:49279 "POST /session HTTP/1.1" 200 882


Navigating to https://www.ted.com/talks?sort=newest


2024-12-06 00:33:31,990 - DEBUG - http://localhost:49279 "POST /session/879878be05151ddbade52f7dc74758a5/url HTTP/1.1" 200 14
2024-12-06 00:33:36,000 - DEBUG - http://localhost:49279 "POST /session/879878be05151ddbade52f7dc74758a5/element HTTP/1.1" 404 1077


  0%|          | 0/300 [00:00<?, ?it/s]

2024-12-06 00:33:37,012 - DEBUG - Starting new HTTPS connection (1): zenith-prod-alt.ted.com:443
2024-12-06 00:33:37,550 - DEBUG - https://zenith-prod-alt.ted.com:443 "POST /api/search HTTP/1.1" 200 None
2024-12-06 00:33:38,553 - DEBUG - Starting new HTTPS connection (1): zenith-prod-alt.ted.com:443
2024-12-06 00:33:38,952 - DEBUG - https://zenith-prod-alt.ted.com:443 "POST /api/search HTTP/1.1" 200 None
2024-12-06 00:33:39,958 - DEBUG - Starting new HTTPS connection (1): zenith-prod-alt.ted.com:443
2024-12-06 00:33:40,338 - DEBUG - https://zenith-prod-alt.ted.com:443 "POST /api/search HTTP/1.1" 200 None
2024-12-06 00:33:41,343 - DEBUG - Starting new HTTPS connection (1): zenith-prod-alt.ted.com:443
2024-12-06 00:33:43,489 - DEBUG - https://zenith-prod-alt.ted.com:443 "POST /api/search HTTP/1.1" 200 None
2024-12-06 00:33:44,493 - DEBUG - Starting new HTTPS connection (1): zenith-prod-alt.ted.com:443
2024-12-06 00:33:44,874 - DEBUG - https://zenith-prod-alt.ted.com:443 "POST /api/search

...

2024-12-06 01:31:45,356 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:31:45,362 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:31:45,844 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:31:45,853 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:31:46,341 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:31:46,346 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:31:46,817 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:31:46,825 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:31:47,332 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:31:47,336 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:31:47,785 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:31:47,788

2024-12-06 01:32:07,483 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:07,926 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:07,935 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:08,374 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:08,380 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:08,818 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:08,821 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:09,282 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:09,293 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:09,805 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:09,811 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:10,265 - DEBU

2024-12-06 01:32:30,003 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:30,012 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:30,607 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:30,612 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:31,089 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:31,094 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:31,574 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:31,577 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:32,048 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:32,053 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:32,547 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:32,558

2024-12-06 01:32:52,872 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:53,316 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:53,322 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:53,835 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:53,838 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:54,305 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:54,311 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:54,781 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:54,790 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:55,323 - DEBUG - https://www.ted.com:443 "POST /graphql HTTP/1.1" 200 None
2024-12-06 01:32:55,326 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 01:32:55,785 - DEBU

Saving data to CSV files...
Scraping completed! Data saved to CSV files.


In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import json

# Load the CSV file with TED talk metadata
df = pd.read_csv("ted_talks_list.csv")  # Use the correct file name

# Ensure the 'transcript' column exists
if "transcript" not in df.columns:
    df["transcript"] = ""  # Create an empty column for transcripts

# Function to extract transcript from TED talk HTML page
def extract_transcript_from_page(url):
    try:
        # Fetch the TED talk page
        response = requests.get(url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        })
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Check for JSON script containing the transcript
            script_tag = soup.find("script", type="application/ld+json")
            if script_tag:
                data = json.loads(script_tag.string)
                transcript = data.get("transcript")
                if transcript:
                    return transcript
            
            # Alternative logic to extract transcript from HTML
            transcript_sections = soup.find_all("div", class_="Grid__cell")
            transcript = " ".join([section.text.strip() for section in transcript_sections])
            if transcript:
                return transcript
        else:
            print(f"Failed to fetch {url}. Status code: {response.status_code}")
            return ""
    except Exception as e:
        print(f"Error fetching transcript for {url}: {e}")
        return ""

# Iterate over the DataFrame and extract transcripts
for index, row in tqdm(df.iterrows(), total=len(df)):
    if pd.isna(row["transcript"]) or not row["transcript"]:  # Skip if transcript already exists
        transcript = extract_transcript_from_page(row["url"])
        df.at[index, "transcript"] = transcript  # Update the DataFrame
    time.sleep(2)  # Delay to avoid server rate-limiting

# Save the updated DataFrame back to a CSV file
df.to_csv("ted_talks_transcripts_updated.csv", index=False)
print("Transcripts saved to 'ted_talks_transcripts_updated.csv'.")


  0%|          | 0/6810 [00:00<?, ?it/s]

2024-12-06 11:45:30,304 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:45:31,469 - DEBUG - https://www.ted.com:443 "GET /talks/andrej_levin_why_do_we_still_negotiate_the_price_of_cars HTTP/1.1" 200 46591
2024-12-06 11:45:33,574 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:45:33,888 - DEBUG - https://www.ted.com:443 "GET /talks/max_g_levy_how_scientists_are_creating_real_life_invisibility_cloaks HTTP/1.1" 200 51774
2024-12-06 11:45:35,964 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:45:37,326 - DEBUG - https://www.ted.com:443 "GET /talks/cara_hunter_how_a_deepfake_almost_ruined_my_political_career HTTP/1.1" 200 47514
2024-12-06 11:45:39,422 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:45:40,697 - DEBUG - https://www.ted.com:443 "GET /talks/nila_ibrahimi_the_song_that_sparked_a_movement_in_afghanistan HTTP/1.1" 200 47819
2024-12-06 11:45:42,779 - DEBUG - Starting new HTTP

Failed to fetch https://www.ted.com/talks/sarah_stroud_and_michael_vazquez_is_safety_worth_the_loss_of_privacy. Status code: 502


2024-12-06 11:47:14,594 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:47:15,848 - DEBUG - https://www.ted.com:443 "GET /talks/neeraj_aggarwal_k_pop_cutting_edge_tech_and_other_ways_asia_is_shaping_the_world HTTP/1.1" 200 58185
2024-12-06 11:47:17,927 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:47:19,636 - DEBUG - https://www.ted.com:443 "GET /talks/sam_kass_a_menu_of_foods_we_might_lose_forever HTTP/1.1" 200 58616
2024-12-06 11:47:21,761 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:47:22,107 - DEBUG - https://www.ted.com:443 "GET /talks/jennifer_golbeck_is_ai_progress_stuck HTTP/1.1" 200 60100
2024-12-06 11:47:24,190 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:47:26,088 - DEBUG - https://www.ted.com:443 "GET /talks/umit_kurt_ugly_history_the_armenian_genocide HTTP/1.1" 200 53734
2024-12-06 11:47:28,182 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024

2024-12-06 11:49:19,777 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:49:21,229 - DEBUG - https://www.ted.com:443 "GET /talks/stephanie_winters_reflections HTTP/1.1" 200 46863
2024-12-06 11:49:23,297 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:49:24,763 - DEBUG - https://www.ted.com:443 "GET /talks/chris_duffy_how_to_be_an_adult_and_how_to_raise_one_w_julie_lythcott_haims HTTP/1.1" 200 48852
2024-12-06 11:49:26,839 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:49:28,233 - DEBUG - https://www.ted.com:443 "GET /talks/giorgia_lupi_what_long_covid_taught_me_about_life_and_data HTTP/1.1" 200 59797
2024-12-06 11:49:30,310 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 11:49:31,720 - DEBUG - https://www.ted.com:443 "GET /talks/ji_hao_the_real_tale_of_the_monkey_king_in_heaven HTTP/1.1" 200 52011
2024-12-06 11:49:33,799 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2

...

2024-12-06 18:32:50,484 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:32:52,964 - DEBUG - https://www.ted.com:443 "GET /talks/rives_if_i_controlled_the_internet HTTP/1.1" 502 107


Failed to fetch https://www.ted.com/talks/rives_if_i_controlled_the_internet. Status code: 502


2024-12-06 18:32:54,967 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:32:55,282 - DEBUG - https://www.ted.com:443 "GET /talks/richard_st_john_8_secrets_of_success HTTP/1.1" 200 54574
2024-12-06 18:32:57,373 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:32:59,007 - DEBUG - https://www.ted.com:443 "GET /talks/dean_ornish_the_killer_american_diet_that_s_sweeping_the_planet HTTP/1.1" 200 54104
2024-12-06 18:33:01,093 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:33:02,224 - DEBUG - https://www.ted.com:443 "GET /talks/peter_gabriel_fight_injustice_with_raw_video HTTP/1.1" 200 58765
2024-12-06 18:33:04,315 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:33:06,692 - DEBUG - https://www.ted.com:443 "GET /talks/ray_kurzweil_the_accelerating_power_of_technology HTTP/1.1" 200 70329
2024-12-06 18:33:08,796 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:33:10

2024-12-06 18:35:02,314 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:35:03,791 - DEBUG - https://www.ted.com:443 "GET /talks/jehane_noujaim_my_wish_a_global_day_of_film HTTP/1.1" 200 69950
2024-12-06 18:35:05,875 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:35:08,166 - DEBUG - https://www.ted.com:443 "GET /talks/larry_brilliant_my_wish_help_me_stop_pandemics HTTP/1.1" 200 70982
2024-12-06 18:35:10,263 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:35:11,675 - DEBUG - https://www.ted.com:443 "GET /talks/dan_dennett_let_s_teach_religion_all_religion_in_schools HTTP/1.1" 200 69802
2024-12-06 18:35:13,769 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:35:15,225 - DEBUG - https://www.ted.com:443 "GET /talks/rick_warren_a_life_of_purpose HTTP/1.1" 200 66408
2024-12-06 18:35:17,312 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-06 18:35:18,669 - DEBUG - htt

Transcripts saved to 'ted_talks_transcripts_updated.csv'.


In [9]:
# Install necessary libraries if not already installed
# !pip install requests beautifulsoup4

import requests
from bs4 import BeautifulSoup

# URL of the TED Talk
url = "https://www.ted.com/talks/bob_mankoff_can_ai_master_the_art_of_humor?subtitle=en"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the script tag containing the transcript JSON
    script_tag = soup.find("script", type="application/ld+json")

    if script_tag:
        import json
        data = json.loads(script_tag.string)

        # Extract the transcript if available
        transcript = data.get("transcript")

        if transcript:
            print("Transcript extracted successfully!")
            print(transcript)
        else:
            print("Transcript not found in the page data.")
    else:
        print("Transcript data not found in the HTML source.")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")


2024-12-05 23:40:48,326 - DEBUG - Starting new HTTPS connection (1): www.ted.com:443
2024-12-05 23:40:48,720 - DEBUG - https://www.ted.com:443 "GET /talks/bob_mankoff_can_ai_master_the_art_of_humor?subtitle=en HTTP/1.1" 200 58917


Transcript extracted successfully!
Whether you&apos;re thrilled by what AI can do for us or terrified by what AI is going to do to us, whether it can be funny, is probably not top of mind for you. It is for me. I don&apos;t care if it turns all of us into paperclips, as long as they&apos;re funny paper clips. (Laughter) And the fact that it makes stuff up, hallucinates, for me, that&apos;s not a bug, that&apos;s a feature. My entire career was making stuff up. They&apos;re called cartoons. This is probably the most famous one I hallucinated. There are a number of theories of humor that could explain this cartoon. There&apos;s the superiority theory. You&apos;re the guy on the phone, not on the other end. The incongruity theory. There&apos;s a mismatch between the politeness of the language and the rudeness of the message. And the benign violation theory of humor, which is sort of a golden ratio theory of humor, if you will. See, I got in that term (Laughter) Which says, for something t